In [2]:
from torch.utils.data import Dataset, DataLoader
import transformers
from torch import nn
from transformers import RobertaForTokenClassification, RobertaTokenizer, RobertaTokenizerFast
from torch.optim import AdamW
import torch
import numpy as np
import glob
from tqdm import tqdm
from sklearn.metrics import f1_score

In [3]:
config = {
    'batch_size': 8,
    'epochs': 60,
    'encoder_final_hidden_state': 768,
    'dropout': 0.2,
    'FILES_PER_BATCH': 2,
    'NUM_TOKENS_TO_PROCESS': 200,
    'lr': 1e-3
}

## Dataset class

In [31]:
class ACLNERDataset(Dataset):

    def __init__(self, path, tokenizer, class_mapping, files_per_batch = 2, num_tokens_to_process = 200):

        files = glob.glob(path)
        file_sizes = []

        self.X_data = []
        self.Y_data = []

        self.files_per_batch = files_per_batch
        self.num_tokens_to_process = num_tokens_to_process
        self.tokenizer = tokenizer
        self.class_mapping = class_mapping
        self.inv_class_mapping = {v:k for k, v in self.class_mapping.items()}

        for i in files:
            
            X = []
            Y = []
            with open(i, 'r', encoding = 'utf-8') as f:
                data = f.readlines()
                file_sizes.append(len(data))
        
                for line in data:
                    line = line.replace('  ', ' ')
                    x, y = line.split(' ')
                    X.append(x)
                    Y.append(y.replace('\n', ''))

                self.X_data.append(X)
                self.Y_data.append(Y)

        seq_len = []
        self.index_map = {}
        k = 0

        for i in range(0, len(files), files_per_batch):
            file_seq_len = 0
            
            for j in range(0, files_per_batch):
                if i+j < len(files):
                    file_seq_len = max(file_seq_len, file_sizes[i+j])

            for j in range(0, files_per_batch):
                if i+j < len(files):
                    difference = file_seq_len - file_sizes[i+j]
                    for num_pads in range(difference):
                        self.X_data[i+j].append("<pad>")
                        self.Y_data[i+j].append("O")

            for j in range(0, file_seq_len + num_tokens_to_process, num_tokens_to_process):
                for m in range(i, i + files_per_batch):
                    if m < len(files):
                        self.index_map[k] = (m, min(j, file_seq_len))
                        k += 1

        self.length = len(self.index_map)


    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        document_set, token_bounds = self.index_map[idx]
        x = self.X_data[document_set][token_bounds: token_bounds+self.num_tokens_to_process+1]
        y = self.Y_data[document_set][token_bounds: token_bounds+self.num_tokens_to_process+1]

        return x, y, document_set

    def collate(self, batch):

        batch_x = [x for x, y, doc in batch]
        batch_y = [y for x, y, doc in batch]
        document = [doc for x, y, doc in batch]

        tokenized_samples = self.tokenizer.batch_encode_plus(batch_x,
                                                             truncation=True,
                                                             padding="max_length",
                                                             max_length = 512,
                                                             return_tensors='pt',
                                                             is_split_into_words=True)
        total_adjusted_labels = []
        for k in range(0, len(tokenized_samples["input_ids"])):
            prev_wid = -1
            word_ids_list = tokenized_samples.word_ids(batch_index=k)
            existing_label_ids = batch_y[k]
            i = -1
            adjusted_label_ids = []

            if existing_label_ids != len(word_ids_list):
                existing_label_ids += ["O"] * (len(word_ids_list) - len(existing_label_ids))

            for wid in word_ids_list:
                if wid is None:
                    adjusted_label_ids.append(self.class_mapping["O"])
                elif wid != prev_wid:
                    i = i + 1
                    adjusted_label_ids.append(self.class_mapping[existing_label_ids[i]])
                    prev_wid = wid
                else:
                    label_name = self.inv_class_mapping[adjusted_label_ids[-1]].replace('B-','I-')
                    # label_name = self.class_mapping[existing_label_ids[i]]
                    # adjusted_label_ids.append(self.class_mapping[existing_label_ids[i]])
                    adjusted_label_ids.append(self.class_mapping[label_name])
            total_adjusted_labels.append(adjusted_label_ids)

        tokenized_samples["labels"] = torch.Tensor(total_adjusted_labels)
        tokenized_samples["document"] = document
        return tokenized_samples

In [151]:
class ACLNERDatasetTest(Dataset):

    def __init__(self, path, tokenizer, files_per_batch = 2, num_tokens_to_process = 200):

        files = glob.glob(path)
        file_sizes = []

        self.X_data = []

        self.files_per_batch = files_per_batch
        self.num_tokens_to_process = num_tokens_to_process
        self.tokenizer = tokenizer

        for i in files:
            
            X = []
            with open(i, 'r', encoding = 'utf-8') as f:
                data = f.readlines()
                file_sizes.append(len(data))
        
                for line in data:
                    
                    line = line.replace('  ', ' ')
                    if line == '\n':
                        x = line
                        y = None
                    else:
                        x,y = line.split(' ')
                    
                    X.append(x)

                self.X_data.append(X)

        seq_len = []
        self.index_map = {}
        k = 0

        for i in range(0, len(files), files_per_batch):
            file_seq_len = 0
            
            for j in range(0, files_per_batch):
                if i+j < len(files):
                    file_seq_len = max(file_seq_len, file_sizes[i+j])

            for j in range(0, files_per_batch):
                if i+j < len(files):
                    difference = file_seq_len - file_sizes[i+j]
                    for num_pads in range(difference):
                        self.X_data[i+j].append("<pad>")

            for j in range(0, file_seq_len + num_tokens_to_process, num_tokens_to_process):
                for m in range(i, i + files_per_batch):
                    if m < len(files):
                        self.index_map[k] = (m, min(j, file_seq_len))
                        k += 1

        self.length = len(self.index_map)


    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        document_set, token_bounds = self.index_map[idx]
        x = self.X_data[document_set][token_bounds: token_bounds+self.num_tokens_to_process+1]
        return x, document_set

    def collate(self, batch):

        batch_x = [x for x, doc in batch]
        tokenized_samples = self.tokenizer.batch_encode_plus(batch_x,
                                                             truncation=True,
                                                             padding="max_length",
                                                             max_length = 512,
                                                             return_tensors='pt',
                                                             is_split_into_words=True)
        all_word_ids_list = []
        for k in range(0, len(tokenized_samples["input_ids"])):
            word_ids_list = tokenized_samples.word_ids(batch_index=k)
            all_word_ids_list.append(word_ids_list)

        tokenized_samples["word_ids"] = all_word_ids_list
        return tokenized_samples

## Dataloaders

In [152]:
class_mapping = {"B-MethodName":0,
                 "I-MethodName":1,
                 "B-HyperparameterName":2,
                 "I-HyperparameterName":3,
                 "B-HyperparameterValue":4,
                 "I-HyperparameterValue":5,
                 "B-MetricName":6,
                 "I-MetricName":7,
                 "B-MetricValue":8,
                 "I-MetricValue":9,
                 "B-TaskName":10,
                 "I-TaskName":11,
                 "B-DatasetName":12,
                 "I-DatasetName":13,
                 "O":14
                }
id_to_class_mapping = {v:k for k, v in class_mapping.items()}

train_data_path = '../data/train/*.conll'
val_data_path = '../data/test/*.conll'
test_data_path = '../data/sciner-test/*.conll'

tokenizer = RobertaTokenizerFast.from_pretrained('ml6team/keyphrase-extraction-kbir-inspec')

train_dataset = ACLNERDataset(train_data_path, tokenizer, class_mapping, config['FILES_PER_BATCH'], config['NUM_TOKENS_TO_PROCESS'])
train_dataloader = DataLoader(train_dataset, batch_size = config['batch_size'], shuffle = False, collate_fn = train_dataset.collate)

val_dataset = ACLNERDataset(val_data_path, tokenizer, class_mapping, config['FILES_PER_BATCH'], config['NUM_TOKENS_TO_PROCESS'])
val_dataloader = DataLoader(val_dataset, batch_size = config['batch_size'], shuffle = False, collate_fn = val_dataset.collate)

test_dataset = ACLNERDatasetTest(test_data_path, tokenizer, config['FILES_PER_BATCH'], config['NUM_TOKENS_TO_PROCESS'])
test_dataloader = DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False, collate_fn = test_dataset.collate)

In [80]:
print(len(val_dataset))
print(len(train_dataset))

142
924


## Initializing the model

In [8]:
class NERModel(nn.Module):
    def __init__(self):
        super(NERModel, self).__init__()
        
        self.encoder = RobertaForTokenClassification.from_pretrained('roberta_mlm_checkpoints/checkpoint-100000')
        self.dropout = nn.Dropout(p=config['dropout'])
        
        self.classification_layer = nn.Sequential(
            nn.Linear(config['encoder_final_hidden_state'], 1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 15)
        )
    
    def forward(self, input_ids, attention_mask, labels, output_hidden_states, requires_grad):
        
        if requires_grad:
            for name, param in self.encoder.named_parameters():
                if '11' not in name:
                    param.requires_grad = False
        else:
            for name, param in self.encoder.named_parameters():
                param.requires_grad = False
                
        encoder_out = self.encoder(input_ids = input_ids, attention_mask = attention_mask, output_hidden_states = True)
        x = self.dropout(encoder_out.hidden_states[-1])
        x = self.classification_layer(x)
        return x, x[:, -1]

In [9]:
model = NERModel()
model = model.cuda()
scaler = torch.cuda.amp.GradScaler()
optimizer = AdamW(params = model.parameters(), lr=config['lr'])
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 100, T_mult = 3, eta_min = 1e-7)

Some weights of the model checkpoint at roberta_mlm_checkpoints/checkpoint-100000 were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta_mlm_checkpoints/checkpoint-100000 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model 

## Training the model

In [12]:
def train(train_dataloader, requires_grad):
    total_train_loss = 0
    model.train()
    prev_doc = None
    batch_bar = tqdm(total=len(train_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    for i, batch in enumerate(tqdm(train_dataloader)):

        optimizer.zero_grad()
        torch.cuda.empty_cache()

        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels']
        labels = labels.type(torch.LongTensor).cuda()
        documents = batch['document']

        with torch.cuda.amp.autocast():
            output= model(input_ids = input_ids, attention_mask = attention_mask, labels = labels, output_hidden_states = True, requires_grad=requires_grad)
            loss = criterion(output[0].reshape(-1, 15), labels.reshape(-1))

        if prev_doc != documents:
            prev_doc = documents
        else:
            last_hidden_state = output

        batch_loss = loss
        total_train_loss += batch_loss
        scaler.scale(loss).backward() 
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        batch_bar.set_postfix(
            loss = f"{total_train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )
        batch_bar.update()
    batch_bar.close()
    avg_train_loss = total_train_loss / len(train_dataloader)
    return avg_train_loss

def val(val_dataloader, val_dataset):
    total_val_loss = 0
    model.eval()
    prev_doc = None
    all_input_id_labels = []
    all_input_id_predictions = []

    batch_bar = tqdm(total=len(val_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Val')
    for i, batch in enumerate(val_dataloader):
        torch.cuda.empty_cache()

        input_ids = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        labels = batch['labels']
        for label in labels.detach().cpu().numpy().flatten():
            all_input_id_labels.append(label)
        labels = labels.type(torch.LongTensor).cuda()
        documents = batch['document']

        with torch.no_grad():
            output= model(input_ids = input_ids, attention_mask = attention_mask, labels = labels, output_hidden_states = True, requires_grad=False)
            predictions = output[0].reshape(-1, 15)
            loss = criterion(predictions, labels.reshape(-1))

        if prev_doc != documents:
            prev_doc = documents
        else:
            last_hidden_state = output
        # for j in range(len(input_ids)):
        #     batch_labels = extract_keyphrases(input_ids[j], predictions.argmax(dim=1), tokenizer, j)
        #     for l in batch_labels:
        #         BOI_predictions.append(class_mapping[l[1]])
        for prediction in predictions.argmax(dim=1).detach().cpu().numpy().flatten():
            all_input_id_predictions.append(prediction)

        batch_loss = loss
        total_val_loss += batch_loss
        batch_bar.set_postfix(
            loss = f"{total_val_loss/ (i+1):.4f}"
        )
        batch_bar.update()
    batch_bar.close()
    
    f1_micro = f1_score(all_input_id_labels, all_input_id_predictions, labels = list(class_mapping.keys()).remove('O'), average='micro')
    f1_macro = f1_score(all_input_id_labels, all_input_id_predictions, labels = list(class_mapping.keys()).remove('O'), average='macro')
    f1_weighted = f1_score(all_input_id_labels, all_input_id_predictions, labels = list(class_mapping.keys()).remove('O'), average='weighted')
    avg_val_loss = total_val_loss / len(val_dataloader)
    return avg_val_loss, f1_micro, f1_macro, f1_weighted

In [13]:
for epoch in range(config['epochs']):
    print("Epoch number", epoch)
    if epoch < 15:
        train_loss = train(train_dataloader, requires_grad = False)
    else:
        train_loss = train(train_dataloader, requires_grad = True)
    print("Train loss", train_loss)
    val_loss, f1_micro, f1_macro, f1_weighted = val(val_dataloader, val_dataset)
    print("Val loss", val_loss)
    print("F-1 score", f1_micro, f1_macro, f1_weighted)

Epoch number 0


Train:   0%|             | 0/116 [00:00<?, ?it/s]
Train:   1%| | 1/116 [00:00<00:41,  2.74it/s, los
Train:   2%| | 2/116 [00:00<00:26,  4.34it/s, los
Train:   3%| | 3/116 [00:00<00:20,  5.43it/s, los
Train:   3%| | 4/116 [00:00<00:18,  6.17it/s, los
Train:   4%| | 5/116 [00:00<00:16,  6.61it/s, los
Train:   5%| | 6/116 [00:01<00:15,  6.97it/s, los
Train:   6%| | 7/116 [00:01<00:15,  7.16it/s, los
Train:   7%| | 8/116 [00:01<00:14,  7.32it/s, los
Train:   8%| | 9/116 [00:01<00:14,  7.48it/s, los
Train:   9%| | 10/116 [00:01<00:14,  7.57it/s, lo
Train:   9%| | 11/116 [00:01<00:16,  6.49it/s, lo
Train:  10%| | 12/116 [00:01<00:15,  6.80it/s, lo
Train:  11%| | 13/116 [00:02<00:14,  7.07it/s, lo
Train:  12%| | 14/116 [00:02<00:14,  7.27it/s, lo
Train:  13%|▏| 15/116 [00:02<00:13,  7.40it/s, lo
Train:  14%|▏| 16/116 [00:02<00:13,  7.54it/s, lo
Train:  15%|▏| 17/116 [00:02<00:13,  7.59it/s, lo
Train:  16%|▏| 18/116 [00:02<00:12,  7.65it/s, lo
Train:  16%|▏| 19/116 [00:02<00:12,  7.66it/s, lo


Train loss tensor(0.2084, device='cuda:0', grad_fn=<DivBackward0>)


Val loss tensor(0.0672, device='cuda:0')
F-1 score 0.9874009683098591 0.06624449504352059 0.9811481782675483
Epoch number 1


Train:   0%|             | 0/116 [00:00<?, ?it/s]
Train:   1%| | 1/116 [00:00<00:14,  8.04it/s, los
Train:   2%| | 2/116 [00:00<00:14,  7.88it/s, los
Train:   3%| | 3/116 [00:00<00:14,  7.83it/s, los
Train:   3%| | 4/116 [00:00<00:14,  7.79it/s, los
Train:   4%| | 5/116 [00:00<00:14,  7.73it/s, los
Train:   5%| | 6/116 [00:00<00:14,  7.71it/s, los
Train:   6%| | 7/116 [00:00<00:14,  7.77it/s, los
Train:   7%| | 8/116 [00:01<00:13,  7.72it/s, los
Train:   8%| | 9/116 [00:01<00:13,  7.73it/s, los
Train:   9%| | 10/116 [00:01<00:13,  7.77it/s, lo
Train:   9%| | 11/116 [00:01<00:13,  7.74it/s, lo
Train:  10%| | 12/116 [00:01<00:13,  7.72it/s, lo
Train:  11%| | 13/116 [00:01<00:13,  7.71it/s, lo
Train:  12%| | 14/116 [00:01<00:13,  7.73it/s, lo
Train:  13%|▏| 15/116 [00:01<00:13,  7.71it/s, lo
Train:  14%|▏| 16/116 [00:02<00:12,  7.76it/s, lo
Train:  15%|▏| 17/116 [00:02<00:12,  7.71it/s, lo
Train:  16%|▏| 18/116 [00:02<00:12,  7.68it/s, lo
Train:  16%|▏| 19/116 [00:02<00:12,  7.68it/s, lo


...

Train loss tensor(0.0215, device='cuda:0', grad_fn=<DivBackward0>)


Val loss tensor(0.0478, device='cuda:0')
F-1 score 0.9872221610915493 0.32922720424308616 0.9845182319039356
Epoch number 59


Train:   0%|             | 0/116 [00:00<?, ?it/s]
Train:   1%| | 1/116 [00:00<00:15,  7.55it/s, los
Train:   2%| | 2/116 [00:00<00:15,  7.47it/s, los
Train:   3%| | 3/116 [00:00<00:15,  7.48it/s, los
Train:   3%| | 4/116 [00:00<00:14,  7.54it/s, los
Train:   4%| | 5/116 [00:00<00:14,  7.49it/s, los
Train:   5%| | 6/116 [00:00<00:14,  7.50it/s, los
Train:   6%| | 7/116 [00:00<00:14,  7.53it/s, los
Train:   7%| | 8/116 [00:01<00:14,  7.53it/s, los
Train:   8%| | 9/116 [00:01<00:14,  7.52it/s, los
Train:   9%| | 10/116 [00:01<00:14,  7.55it/s, lo
Train:   9%| | 11/116 [00:01<00:14,  7.49it/s, lo
Train:  10%| | 12/116 [00:01<00:13,  7.46it/s, lo
Train:  11%| | 13/116 [00:01<00:13,  7.48it/s, lo
Train:  12%| | 14/116 [00:01<00:13,  7.48it/s, lo
Train:  13%|▏| 15/116 [00:01<00:13,  7.50it/s, lo
Train:  14%|▏| 16/116 [00:02<00:13,  7.52it/s, lo
Train:  15%|▏| 17/116 [00:02<00:13,  7.50it/s, lo
Train:  16%|▏| 18/116 [00:02<00:13,  7.47it/s, lo
Train:  16%|▏| 19/116 [00:02<00:12,  7.47it/s, lo


Train loss tensor(0.0210, device='cuda:0', grad_fn=<DivBackward0>)


Val loss tensor(0.0482, device='cuda:0')
F-1 score 0.9874697403169014 0.32299296804784944 0.984632166351046


In [14]:
torch.save(model, 'model.pth')

## Test the model

In [153]:
def test(test_dataloader):
    total_test_loss = 0
    model.eval()
    all_words = []
    all_word_predictions = []

#     batch_bar = tqdm(total=len(test_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Test')
    for i, batch in enumerate(test_dataloader):
        torch.cuda.empty_cache()

        input_ids = batch['input_ids'].cuda()
        
        attention_mask = batch['attention_mask'].cuda()
        all_word_ids_list = batch['word_ids']

        with torch.no_grad():
            output= model(input_ids = input_ids, attention_mask = attention_mask, output_hidden_states = True, labels = None, requires_grad=False)
            predictions = output[0]

        for j, document_wise_predictions in enumerate(predictions):
            one_document_prediction = document_wise_predictions.argmax(-1)
            document_input_ids = input_ids[j]
            prev_word_id = -1
            temp_word_ids = []
            current_label = 'O'
            k = -1
            for word_id, prediction in zip(all_word_ids_list[j], one_document_prediction):
                if word_id == None:
                    k+=1
                    continue
                
                if prev_word_id == word_id:
                    k += 1
                    temp_word_ids.append(document_input_ids[k].item())
                else:
                    if len(temp_word_ids) > 0:
                        all_words.append(tokenizer.decode(temp_word_ids, skip_special_tokens = True))
                        all_word_predictions.append(current_label)
                        temp_word_ids = []
                        
                    current_label = id_to_class_mapping[prediction.item()]
                    k += 1
                    temp_word_ids.append(document_input_ids[k].item())
                    prev_word_id = word_id
                    
    return all_words, all_word_predictions

In [154]:
all_words, all_word_predictions = test(test_dataloader)

with open('predictions.conll', 'w') as f:
    for i,j in zip(all_words, all_word_predictions):
        if i.strip() == '':
            f.write('\n')
        else:
            f.write(i.strip() + ' ' + j + '\n')

In [150]:
!pip3 install --upgrade --force-reinstall explainaboard_client==0.0.9 explainaboard_api_client==0.2.8

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Using cached explainaboard_client-0.0.9-py2.py3-none-any.whl (19 kB)
  Using cached explainaboard_api_client-0.2.8-py2.py3-none-any.whl (173 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


In [144]:
!export EB_USERNAME="pkarnani@andrew.cmu.edu"
!export EB_API_KEY="CT2Mr-yEXXek124uuO8sHg"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [146]:
!python3 -m explainaboard_client.cli.evaluate_system \
  --username $EB_USERNAME \
  --api-key $EB_API_KEY \
  --task named-entity-recognition \
  --system-name robertamlmner \
  --dataset cmu_anlp \
  --sub-dataset sciner \
  --split test \
  --system-output-file anlp_pkarnani_robertamlmner.conll \
  --system-output-file-type conll \
  --shared-users neubig@gmail.com \
  --source-language en

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "/usr/lib64/python3.7/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib64/python3.7/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/home/ec2-user/.local/lib/python3.7/site-packages/explainaboard_client/__init__.py", line 3, in <module>
    from typing import Literal
ImportError: cannot import name 'Literal' from 'typing' (/usr/lib64/python3.7/typing.py)


In [ ]:
for i in 